In [1]:
from datetime import datetime, timedelta
import os
import warnings
import time
import json
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
from owslib.wms import WebMapService
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
wms = WebMapService('https://geo.weather.gc.ca/geomet?SERVICE=WMS' +
                    '&REQUEST=GetCapabilities',
                    version='1.3.0',
                    timeout=300)

In [3]:
timeseriesfolder = "2023_RDPA"
start_time = '2023-01-01T12:00'
end_time = '2023-12-31T12:00'
total_precip_shapefile = 'CaPA_grid_total_precip_2023-2023.shp'

In [4]:
interval = 24
fmt = '%Y-%m-%dT%H:%M'
start_time = datetime.strptime(start_time, fmt)
end_time = datetime.strptime(end_time, fmt)

timesteps = [start_time]
while timesteps[-1] < end_time:
    timesteps.append(timesteps[-1] + timedelta(hours=interval))

In [5]:
station = '07DD001'
layer = 'RDPA.24F_PR'  # 24-hour precipitation accumulation
crs = 'EPSG:4326'
crs2 = 'ESRI:102001'
folder = r'C:\Users\LeachJ\Documents\GIS DataBase\HydrometricNetworkBasinPolygons'
shapefile = os.path.join(folder, station[:2], station, f"{station}_DrainageBasin_BassinDeDrainage.shp")
if not os.path.exists(shapefile):
    shapefile = os.path.join(folder, station[:2], f'EC_{station}_1.shp')
if os.path.exists(shapefile):
    gdf_boundary = gpd.read_file(shapefile, engine='pyogrio', use_arrow=True).to_crs(crs2)
gdf_boundary = gdf_boundary.buffer(7000)  # 7 km buffer
gdf_boundary2 = gdf_boundary.to_crs(crs)

In [6]:
bbox = tuple(gdf_boundary2.total_bounds)

In [7]:
# print(wms[layer].boundingBoxWGS84)
# print(wms[layer].dimensions)

In [8]:
# Loop to carry out the requests and extract the probabilities
def request(layer, timesteps, crs, bbox, feature_count, outfolder): 
    if not os.path.exists(outfolder):
        os.mkdir(outfolder)
    for timestep in timesteps:
        outputfile = os.path.join(outfolder, f"{timestep.strftime('%Y-%m-%d')}.shp")
        if not os.path.exists(outputfile):
            # WMS GetFeatureInfo query
            json_data = wms.getfeatureinfo(layers=[layer],
                                           srs=crs,
                                           bbox=bbox,
                                           size=(30, 30),  # needed manual adjustment
                                           format='image/jpeg',
                                           query_layers=[layer],
                                           info_format='application/json',
                                           xy=(15, 25),  # needed manual adjustment
                                           feature_count=feature_count,
                                           time=str(timestep.isoformat()) + 'Z')
            try:
                features = json.loads(json_data.read().decode('utf-8'))['features']
                geomlst = []
                valulst = []
                for feature in features:
                    geomlst.append(feature['geometry'])
                    valulst.append(feature['properties']['value'])
        
                gdf = gpd.GeoDataFrame()
                geom = [shape(i) for i in geomlst]
                gdf['values'] = np.float32(valulst)
                idx = gdf['values'] == 9999.0
                gdf['values'][idx] = np.nan
                gdf = gdf.set_geometry(geom).set_crs(crs)
                gdf.to_file(outputfile, engine='pyogrio', use_arrow=True)
            except KeyError:
                print(timestep, json.loads(json_data.read().decode('utf-8')))

tic = time.time()
request(layer, timesteps, crs, bbox, 8000, timeseriesfolder)
toc = time.time()
print(toc-tic)

2023-01-07 12:00:00 {}
2023-03-11 12:00:00 {}
2023-03-12 12:00:00 {}
2023-03-13 12:00:00 {}
2023-03-14 12:00:00 {}
2023-08-01 12:00:00 {}
2023-09-04 12:00:00 {}
4.27522611618042


In [9]:
tic = time.time()
for i, timestep in enumerate(timesteps):
    shpfile = os.path.join(timeseriesfolder, f"{timestep.strftime('%Y-%m-%d')}.shp")
    if os.path.exists(shpfile):
        if i == 0:
            gdf = gpd.read_file(shpfile, engine='pyogrio', use_arrow=True)
        else:
            gdf_tmp = gpd.read_file(shpfile, engine='pyogrio', use_arrow=True)
            gdf['values'] += gdf_tmp['values']
toc = time.time()
print(toc-tic)

73.78275871276855


In [10]:
gdf = gpd.clip(gdf.to_crs(gdf_boundary.crs), gdf_boundary)

In [11]:
out_folder = r'C:\Users\LeachJ\Documents\06_Oil Sands Monitoring Group\gis'
gdf.to_file(os.path.join(out_folder, total_precip_shapefile))

In [12]:
flow_stn = ['07BE001', '07DD001', '07DA001', '07DA018', '07DA040', '07DA033', '07CE013', '07CE002', '07CE007',
            '07CD005', '07CD001', '07DB002', '07DB003', '07DA038', '07DA039', '07DA032', '07DA041', '07DC001',
            '07DC003', '07CE008', '07CD004', '07CD008', '07CD009', '07CB002', '07DA027', '07CE005', '07DA026',
            '07DA030', '07DB006', '07DB001', '07DC004', '07DA035', '07DA029', '07DA028', '07DA008', '07DA034',
            '07CE003', '07DA007', '07DA042', '07DA044', '07DA006', '07CE010', '07DA037', '07DA045']

level_stn = ['07CE001', '07DA024', '07DA023', '07DA025']

stations = [*flow_stn, *level_stn]

In [13]:
data_dict = {}
tic = time.time()
for timestep in timesteps:
    shpfile = os.path.join(timeseriesfolder, f"{timestep.strftime('%Y-%m-%d')}.shp")
    if os.path.exists(shpfile):
        data_dict[timestep.strftime('%Y-%m-%d')] = gpd.read_file(shpfile, engine='pyogrio', use_arrow=True).to_crs(crs2)
    else:
        data_dict[timestep.strftime('%Y-%m-%d')] = np.nan
toc = time.time()
print(toc-tic)

92.23139548301697


In [14]:
df = pd.DataFrame()
df['datetime'] = pd.to_datetime(timesteps)
tic = time.time()
for station in stations:
    shapefile = os.path.join(folder, station[:2], station, f"{station}_DrainageBasin_BassinDeDrainage.shp")
    if not os.path.exists(shapefile):
        shapefile = os.path.join(folder, station[:2], f'EC_{station}_1.shp')
    if os.path.exists(shapefile):
        gdf_basin = gpd.read_file(shapefile, engine='pyogrio', use_arrow=True).to_crs(crs2)
        gdf_basin = gdf_basin.buffer(7000)  # 7 km buffer
        
    timeseries = []
    for timestep in timesteps:
        try:
            gdf_tmp = gpd.clip(data_dict[timestep.strftime('%Y-%m-%d')], gdf_basin)
            timeseries.append(np.mean(gdf_tmp['values']))  # equal weights to each grid... not exactly accurate
        except TypeError:  # catch when data is missing / was replaced with nan
            timeseries.append(data_dict[timestep.strftime('%Y-%m-%d')])

    df[station] = timeseries
        
toc = time.time()
print(toc-tic)

81.70107793807983


In [15]:
df.set_index('datetime').to_csv(f"RDPAPrecip{np.unique(df['datetime'].dt.year)[0]}.csv")

In [16]:
tic = time.time()
df_summary = pd.DataFrame()
df_summary['StationID'] = stations
df_summary['YEAR'] = np.unique(df['datetime'].dt.year)[0]

tot_lst = []
mis_lst = []
for station in stations:
    tot_lst.append(np.nansum(df[station]))
    mis_lst.append(np.sum(np.isnan(df[station])))

df_summary['VALUE'] = tot_lst
df_summary['MISSING'] = mis_lst

df_summary.to_csv(f"CaPA_MAP_{df_summary['YEAR'][0]}-{df_summary['YEAR'][0]}.csv", index=False)

toc = time.time()
print(toc-tic)

0.017035961151123047
